# Transfer Learning with TensorFlow Part 1: Feature Extraction

### Build and fit a model using the same data used in main notebook but with MobileNetV2 architecture feature extraction [MobileNetV2](https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5) from TensorFlow Hub. How does it perform compared to the other models trained in main notebook?

In [9]:
# Let's check the structure and contains of each folder
import os

dataset_path = "04_exercise_dataset/"

for dirpath, dirnames, filenames in os.walk(dataset_path):
	print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}")

There are 2 directories and 1 images in 04_exercise_dataset/
There are 0 directories and 3 images in 04_exercise_dataset/model_ckpt
There are 2 directories and 0 images in 04_exercise_dataset/10_food_classes_10_percent
There are 10 directories and 0 images in 04_exercise_dataset/10_food_classes_10_percent/test
There are 0 directories and 250 images in 04_exercise_dataset/10_food_classes_10_percent/test/ice_cream
There are 0 directories and 250 images in 04_exercise_dataset/10_food_classes_10_percent/test/chicken_curry
There are 0 directories and 250 images in 04_exercise_dataset/10_food_classes_10_percent/test/steak
There are 0 directories and 250 images in 04_exercise_dataset/10_food_classes_10_percent/test/sushi
There are 0 directories and 250 images in 04_exercise_dataset/10_food_classes_10_percent/test/chicken_wings
There are 0 directories and 250 images in 04_exercise_dataset/10_food_classes_10_percent/test/grilled_salmon
There are 0 directories and 250 images in 04_exercise_datas

In [10]:
# Setup the data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Setup global variables
IMAGE_SHAPE = (224, 224) # H x W x CC
BATCH_SIZE = 8
EPOCHS = 3

train_dir = "04_exercise_dataset/10_food_classes_10_percent/train/"
test_dir = "04_exercise_dataset/10_food_classes_10_percent/test/"

# Rescale the images
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

# Run the pipelines
print("Training images: ")
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               shuffle=True,
                                               class_mode='categorical',
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE)

print("Test data: ")
test_data = test_datagen.flow_from_directory(directory=test_dir,
                                             shuffle=False,
                                             class_mode='categorical',
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE)

Training images: 
Found 750 images belonging to 10 classes.
Test data: 
Found 2500 images belonging to 10 classes.


In [11]:
# Create callbacks for training
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define callbacks
import datetime
creation_time = datetime.datetime.now().strftime("%Y%m%d - %H%M%S")

checkpoint_dir = f"04_exercise_dataset/model_ckpt/{creation_time}"
model_ckpt = ModelCheckpoint(filepath=checkpoint_dir,
                             monitor='val_loss',
                             save_weights_only=True,
                             verbose=1)

early_stop = EarlyStopping(monitor='val_loss', # maybe add start_from_epoch value for warm-up period?
                           patience=2,
                           verbose=1,
                           restore_best_weights=True)

In [12]:
# Define path for model to transfer
mobilenet_v2 = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"

# Import necessary tools
import tensorflow as tf
import tensorflow_hub as hub

In [13]:
# Creating model for URL
feature_extraction_layer = hub.KerasLayer(handle=mobilenet_v2,
                                          trainable=False,
                                          name='feature_extraction',
                                          input_shape=IMAGE_SHAPE+(3,))

model = tf.keras.Sequential([
    feature_extraction_layer,
    tf.keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_extraction (KerasLa  (None, 1280)             2257984   
 yer)                                                            
                                                                 
 dense_1 (Dense)             (None, 10)                12810     
                                                                 
Total params: 2,270,794
Trainable params: 12,810
Non-trainable params: 2,257,984
_________________________________________________________________


In [14]:
# Compile the model
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit the model
history_model = model.fit(train_data,
                          epochs=15,
                          steps_per_epoch=len(train_data),
                          validation_data=test_data,
                          validation_steps=int(0.1*len(test_data)),
                          callbacks=[early_stop, model_ckpt])

# Evaluate
model.evaluate(test_data)

Epoch 1/15


2023-07-07 23:22:44.090380: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


94/94 [==============================] - ETA: 0s - loss: 1.5298 - accuracy: 0.5067

2023-07-07 23:22:47.944853: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: saving model to 04_exercise_dataset/model_ckpt/20230707 - 232239
94/94 [==============================] - 7s 54ms/step - loss: 1.5298 - accuracy: 0.5067 - val_loss: 1.5962 - val_accuracy: 0.4113
Epoch 2/15
93/94 [============================>.] - ETA: 0s - loss: 0.6757 - accuracy: 0.8181
Epoch 2: saving model to 04_exercise_dataset/model_ckpt/20230707 - 232239
94/94 [==============================] - 4s 45ms/step - loss: 0.6792 - accuracy: 0.8173 - val_loss: 0.9944 - val_accuracy: 0.6411
Epoch 3/15
93/94 [============================>.] - ETA: 0s - loss: 0.4744 - accuracy: 0.8774
Epoch 3: saving model to 04_exercise_dataset/model_ckpt/20230707 - 232239
94/94 [==============================] - 4s 45ms/step - loss: 0.4727 - accuracy: 0.8787 - val_loss: 0.8279 - val_accuracy: 0.7177
Epoch 4/15
93/94 [============================>.] - ETA: 0s - loss: 0.3404 - accuracy: 0.9272
Epoch 4: saving model to 04_exercise_dataset/model_ckpt/20230707 - 232239
94/94 [========================

[0.6206080913543701, 0.8011999726295471]

Model trained on **MobileNetV2** was able to achieve 0.599 loss and 80.1% accuracy on the test set in 6 epochs. Comparing it to the previous models (from the main notebook):
* **ResNET V50** - loss: 0.651, acccuracy: 77.6% in 5 epochs
* **EfficientNetB0** - loss: 0.406, accuracy: 87.9% in 15 epochs

Given the mentioned results, MobileNetV2 is comparable to ResNET V50 architecture having approx. 10 times less parameters (2.27M vs 23.5M) and taking less time per epoch averaging at 5 seconds versus 18 seconds for the latter.

EfficientNetB0 however seems to be a clear winner with close to 10% better accuracy, 50% lower loss metric at approx. 4.04M parameters and epoch time averaging at 15 seconds.

# Build a model to classify images of two different thing you've taken photos of.
* You can usse any feature extraction layer from TensorFlow Hub you like for this.
* You should aim to have at least 10 images of each class.
* Compare its performance to previous models.